In [1]:
import pickle
from geocoding_getter import geocoder
import numpy as np
import pandas as pd

# Please don't simply run all code blocks; some will overwrite existing cached geocodings!! (Even though we have git to back us up)

In [2]:
geocodings = dict()

In [ ]:
with open("../data/geocodings.pickle", 'wb') as f:
    pickle.dump(geocodings, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open("../data/geocodings.pickle", 'rb') as f:
    geocodings = pickle.load(f)

In [3]:
DATA_FOLDER = "../data/raw_mobibikes_data/"
PREFIX = "Mobi_System_Data_"
VERBOSE = True

In [4]:
test_file = "2017.xlsx"
df = pd.read_excel(DATA_FOLDER + PREFIX + test_file)

In [5]:
def get_geocoding(station):
    if VERBOSE:
        print("looking for ... " + station)
    
    if station in geocodings:
        return geocodings[station]
    else:
        lookup_res = geocoder(station)
        geocodings[station] = lookup_res
        
        return lookup_res

In [6]:
def apply_geocoding(row):
    _type = ["Departure", "Return"]
    
    for t in _type:
        station_name = row[f"{t} station"]
        
        if not isinstance(station_name, str):
            values = (None, None, None)
        else:
            values = get_geocoding(station_name)
        
        tmp_series = pd.Series(data=values, 
                               index=[f"{t} postal code",
                                      f"{t} lat",
                                      f"{t} long"])
        
        row = row.append(tmp_series)

    return row

In [ ]:
geocoded_df = df.apply(apply_geocoding, axis=1)

looking for ... 0048 Comox & Broughton
looking for ... 0002 Burrard Station
looking for ... 0048 Comox & Broughton
looking for ... 0048 Comox & Broughton
looking for ... 0093 Chilco & Robson
looking for ... 0035 Coal Harbour Community Centre
looking for ... 0071 14th & Granville
looking for ... 0211 14th & Alder
looking for ... 0187 Bute & Davie
looking for ... 0079 Nelson & Richards
looking for ... 0114 Cypress & 14th
looking for ... 0068 Cypress & Broadway
looking for ... 0211 14th & Alder
looking for ... 0071 14th & Granville
looking for ... 0036 Bute & Robson
looking for ... 0032 Comox & Denman
looking for ... 0024 Hornby & Pender
looking for ... 0037 Hornby & Nelson
looking for ... 0099 Vancouver Art Gallery - North Plaza
looking for ... 0032 Comox & Denman
looking for ... 0024 Hornby & Pender
looking for ... 0016 Pender & Burrard
looking for ... 0024 Hornby & Pender
looking for ... 0016 Pender & Burrard
looking for ... 0024 Hornby & Pender
looking for ... 0030 Abbott & Cordova
lo

In [ ]:
geocoded_df.to_csv("../tmp_2017_geocoded.csv")

In [ ]:
# Check script
for k, v in geocodings.items():
    if len(v[0]) != 6:
        print(k, v)